In [1]:
from making_table_helper import *

In [2]:
import requests
import json
from tqdm import tqdm
import os
import yaml
from multiprocessing import Pool
import itertools
from web3 import Web3
import re
from cryptography.fernet import Fernet
import random

def load_data_structure(read_from_file_name):
    assert read_from_file_name.endswith('.yaml')
    with open(read_from_file_name, 'r') as file:
        data = yaml.safe_load(file)
    return data


def read_config_d():
    config_d = load_data_structure('./config.yaml')
    
    # To use the crypto stuff:
    # cipher_text = cipher_suite.encrypt(text)
    # plain_text = cipher_suite.decrypt(cipher_text)
    cipher_suite = Fernet(config_d['enc_key'])

    enc_key = config_d['enc_key']
    for k in config_d:
        if k == 'enc_key':
            continue
        else:
            config_d[k] = cipher_suite.decrypt(config_d[k])
    
    config_d['github_username'] = config_d['github_username'].decode()
    config_d['etherscan_key'] = config_d['etherscan_key'].decode()

    return config_d

config_d = read_config_d()

In [3]:


def call_Etherscan_API_single(url_params):
    api_url = "http://api.etherscan.io/api"

    url_response = requests.get(api_url, params=url_params, timeout=100, headers={'User-Agent': 'Chrome'})
    result_data = url_response.text

    print(result_data)
    assert json.loads(result_data)["status"] == "1"
#     return result_data["result"]
    return result_data



def write_to_file(contents, path):
    with open(path, 'w') as f:
        f.write(contents)
    
def read_from_file(path):
    with open(path) as f:
        return ''.join(f.readlines())


def hash_file_exists_in_cache(type_op, hash_num):
    # name_no_txt is either transaction or contract hash
    target_path = './etherscan_cache/' + type_op + '/'  + hash_num + '.txt'
    assert not hash_num.endswith('.txt') and hash_num.startswith('0x')
    
    return os.path.exists(target_path)

def read_hash_file_from_cache(type_op, hash_num):
    # name_no_txt is either transaction or contract hash
    target_path = './etherscan_cache/' + type_op + '/'  + hash_num + '.txt'
    assert not hash_num.endswith('.txt') and hash_num.startswith('0x')
    
    assert os.path.exists(target_path)
    return read_from_file(target_path)



def write_hash_file_to_cache(type_op, hash_num, contents):
    # name_no_txt is either transaction or contract hash
    target_path = './etherscan_cache/' + type_op + '/'  + hash_num + '.txt'
    assert not hash_num.endswith('.txt') and hash_num.startswith('0x')
    
    assert not os.path.exists(target_path)
    return write_to_file(contents, target_path)



    
def get_contract_source_code(address):
    assert address.startswith('0x')
    url_params = {
        "module" : "contract",
        "action" : "getsourcecode",
        "apikey" : config_d['etherscan_key'],
        "address" : address
    }
    
    if hash_file_exists_in_cache('getsourcecode', address):
        return read_hash_file_from_cache("getsourcecode", address)
    
    
    response = call_Etherscan_API_single(url_params)
    
    write_hash_file_to_cache('getsourcecode', address, response)

    return response

In [4]:
import re
from collections import defaultdict

# File path to the text file containing hashes
file_folder = './etherscan_cache/vyper_recent_contracts/'
all_vyper_hashes = []

for single_file in os.listdir(file_folder):
    if single_file.startswith('.'):
        continue
    file_path = os.path.join(file_folder, single_file)
    
    # Dictionary to store hash occurrences
    hash_occurrences = defaultdict(int)

    # Regular expression pattern to match hexadecimal hashes
    # This pattern matches strings that start with "0x" or "Q_0x"
    # followed by one or more hexadecimal characters
    hash_pattern = re.compile(r'0x[0-9a-fA-F]+')

    # Read the file and count hash occurrences
    with open(file_path, 'r') as file:
        for line in file:
            # Find all matches of the hash pattern in the line
            matches = hash_pattern.findall(line)
            # Update the occurrences of each hash in the dictionary
            for match in matches:
                # Concatenate the match parts to form the full hash
                full_hash = ''.join(match)
                hash_occurrences[full_hash.lower()] += 1

    # Convert defaultdict to a regular dict for the final result
    hash_occurrences = dict(hash_occurrences)

    hash_occurrences = {k:v for k,v in hash_occurrences.items() if v == 5}

    print(len(hash_occurrences))
    all_vyper_hashes.extend(list(hash_occurrences.keys()))
#     # Print the dictionary mapping hash to the number of occurrences
#     for hash_code, count in hash_occurrences.items():
#         print(f"Hash: {hash_code}, Occurrences: {count}")

100
100
100
100
100


In [5]:
len(all_vyper_hashes)

500

In [8]:
for existing_hash in all_vyper_hashes[:]:
    cur_result = json.loads(get_contract_source_code(existing_hash))['result']
    print("vyper hash: ", existing_hash)


vyper hash:  0xe2554caa829acace7042a3808275dfdb97a6187c
vyper hash:  0x85bd835fb791f96a6d96b902a41079a600abfff9
vyper hash:  0x1269b83b94840d8a84436aff07ad32d36477dd93
vyper hash:  0x827b8f142a54cc901a626911f4934a75a292f687
vyper hash:  0xb6cb6af08ab8f9bd7a8dfc0ea3fc0dbe92b68663
vyper hash:  0x0d7e41242edd789ac5da8828f94642ec958afe56
vyper hash:  0x5005e71f10d006b087f5deb48f24480824d3a80b
vyper hash:  0x33127c982cd73ef4a22b0f2e1f980382ccb7d97b
vyper hash:  0x9b2469abe3f0adff7d3e1c7a8e4215995a47316c
vyper hash:  0xd27b00b6386172fc915f79a06b1374df2da57d19
vyper hash:  0x525e99f743ee495f3b032baa737bc590dc1c85b5
vyper hash:  0x7d4ee86e7d1f9604a968ccbea0e91616fa33555c
vyper hash:  0xad0c09fd5080c1458d2eaa0e44787738eda2ccbb
vyper hash:  0x61c8a978e078a03c671303cc521d31bdd0a4df87
vyper hash:  0x8d69e9bd46d3234a43fac3861b2a591c23546ec2
vyper hash:  0xf68513fc61a040a29f3947ffff47a42e7c81082b
vyper hash:  0x82290330449770fc0bf786a2118b1e2119ebd5da
vyper hash:  0xc1880758d240a95a806ff4f61c912e866

vyper hash:  0xb3c8ce1ee157b0dcaa96897c9170aee6281706c9
vyper hash:  0x112e37742015ece4ceb4b576a9434940838eaf02
vyper hash:  0xde31c340545c8031843bff5eb42640009961aeef
vyper hash:  0xa6c73dc07e17feda6925c2a4f44c166fc18fcf1f
vyper hash:  0x40a442f8cbfd125a762b55f76d9dba66f84dd6dd
vyper hash:  0x8b6527063fbc9c30731d7e57f1def08edce57d07
vyper hash:  0x55a3a857598b500fd844610ed875ec73ff296c4b
vyper hash:  0x1b2540d9f6dea6c42dddfa86be95d5baddb809ec
vyper hash:  0x232ca39b6271b4f896258805021b8629b8006a0f
vyper hash:  0xd4dc9d7567f76fad2b5a90d3a1bdb3ee801435a8
vyper hash:  0xc510d73ad34bedeca8978b6914461aa7b50cf3fc
vyper hash:  0xe7b1c8cfc0bc45957320895aa06884d516daa8e6
vyper hash:  0xe0a4c53408f5acf3246c83b9b8bd8d36d5ee38b8
vyper hash:  0xeda215b7666936ded834f76f3fbc6f323295110a
vyper hash:  0xafca625321df8d6a068bdd8f1585d489d2acf11b
vyper hash:  0xef42b6525454daba2d73441a5c749c82a942692b
vyper hash:  0x413fd2511bad510947a91f5c6c79ebd8138c29fc
vyper hash:  0x5338b1bf469651a5951ef618fb5defbff